In [7]:
from typing import TypedDict,Annotated,Sequence
from langchain_core.messages import BaseMessage,ToolMessage,SystemMessage,HumanMessage
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from pathlib import Path
import os

In [17]:
dotenv_path = Path('.env')
load_dotenv(dotenv_path=dotenv_path)
api_key=os.getenv("gptAPIKEY")
# print(api_key)

In [9]:
document_content = ""

class Agent(TypedDict):
    message : Annotated[Sequence[BaseMessage], add_messages]

@tool
def update(content : str) -> str:
    "update the document with provided content"
    global document_content
    document_content = content
    return f"Document has been updated successfully! The current content is:\n{document_content}"

@tool
def save(filename: str)->str:
    '''save the current document to a textfile and finish the process.
        Args:
        filename: Name for the text file.
    '''
    if not filename.endswith('.txt'):
        filename=f"{filename}.txt" 
    
    global document_content

    try:
        with open(filename,"w") as file:
            file.write(document_content)
        print("i")
        return "contents has been sucessfully been saved in "+filename

    except Exception as e:
        return "couldn't save the contents in "+filename
    
tools = [update,save]

In [10]:
llm= ChatOpenAI(model="z-ai/glm-4.5-air:free",api_key=api_key,base_url="https://openrouter.ai/api/v1").bind_tools(tools)

In [11]:
def ouragent(state:Agent)->Agent:
    system_prompt = SystemMessage(content=f"""
    You are Drafter, a helpful writing assistant. You are going to help the user update and modify documents.
    
    - If the user wants to update or modify content, use the 'update' tool with the complete updated content.
    - If the user wants to save and finish, you need to use the 'save' tool.
    - Make sure to always show the current document state after modifications.
    
    The current document content is:{document_content}
    """)

    if not state['message']:
        userinput="I'm ready to help you update a document. What would you like to create?"
        usermessage=HumanMessage(content=userinput)
    else:
        userinput=input("\nWhat would you like to do with the document")
        print(f"\n User={userinput}")
        usermessage=HumanMessage(content=userinput)

    all_message = [system_prompt]+ list(state["message"])+[usermessage]

    response= llm.invoke(all_message) 

    print('AI ',response.content)

    return {'message': list(state['message'])+[userinput,response]}

def shouldcontinue(state:Agent)->str:
    'Determine if we should continue or end the conversation'
    messages = state['message']
    if not messages:
        return "continue"
    
    for i in reversed(messages):
        if (isinstance(messages,ToolMessage) and "saved" in messages.content.lower() and 'document' in messages.content.lower()):
            return "end" 
    
    return "continue"

In [12]:
graph = StateGraph(Agent)
graph.add_node("agent",ouragent)
graph.add_node("tools",ToolNode(tools=tools))
graph.set_entry_point("agent")
graph.add_edge("agent","tools")
graph.add_conditional_edges(
    "tools",
    shouldcontinue,
    {
        "continue":"agent",
        "end":END,
    }
)
app=graph.compile()

In [16]:
state={'message':[]}
# for i in app.stream(state,stream_mode="values"):
    # if  'message' in i:
        # print(i['message'])